In [46]:
import h5py
import scipy
import numpy as np
import os
import zlib
import msgpack
from sklearn.utils import sparsefuncs
import pickle

def read_compressed_json(json_path):
	if not os.path.isfile(json_path):
		return {}
	with open(json_path, 'rb') as f:
		o = f.read()
		o = zlib.decompress(o)
		o = msgpack.unpackb(o, strict_map_key=False)
		return o

In [47]:
GENES_ANNOTATION = read_compressed_json('/home/ub-sonvo-25d094476064960/genes_annotation.json.gz')
N_GENES = GENES_ANNOTATION['n_genes']
DEFAULT_SIZE_FACTOR = 10000
ROOT_DIR = '/home/ub-sonvo-25d094476064960/celltype_prediction'
# TRAINED_STUDIES = open(ROOT_DIR + '/studies_idx_HVG.txt').read().splitlines()
TRAINED_STUDIES = sorted(os.listdir('/home/ub-sonvo-25d094476064960/celltype_prediction/camel_2'))

In [59]:
def _normalize_total(csr_matrix, counts):
	csr_matrix = csr_matrix.astype(np.float32)
	counts += counts == 0
	counts = counts / 10000
	sparsefuncs.inplace_row_scale(csr_matrix, 1 / counts)
	return csr_matrix


def normalize_total(csr_cxg, n_genes=N_GENES):
    csr_cxg = csr_cxg[:, :n_genes]
    counts_per_cell = csr_cxg.sum(1)
    counts_per_cell = np.ravel(counts_per_cell)

    csr_cxg = _normalize_total(csr_cxg, counts_per_cell)
    csr_cxg.data = np.log1p(csr_cxg.data)

    return csr_cxg


def convert_library_size(csr_cxg, n_genes=N_GENES, size_factor=DEFAULT_SIZE_FACTOR):
    res = []
    for i in range(50):
        res.append(
            np.sum(
                np.expm1(
                    csr_cxg.data[csr_cxg.indptr[i] : csr_cxg.indptr[i+1]]
                )
            )
        )
    res = [np.round(i) for i in res]
    ori_size_factor = np.unique(res)
    upper_bound = ori_size_factor[0] + ori_size_factor[0] * 0.1
    lower_bount = ori_size_factor[0] - ori_size_factor[0] * 0.1

    if np.all(ori_size_factor > lower_bount) and np.all(ori_size_factor < upper_bound):
        csr_cxg = csr_cxg[:, :n_genes]
        if DEFAULT_SIZE_FACTOR < upper_bound and DEFAULT_SIZE_FACTOR > lower_bount:
            return csr_cxg
            
        csr_cxg.data = np.expm1(csr_cxg.data)
        csr_cxg.data = csr_cxg.data * (DEFAULT_SIZE_FACTOR / ori_size_factor[0])
        csr_cxg.data = np.log1p(csr_cxg.data)
        return csr_cxg
        
    print('cannot find the library size:', ori_size_factor)
    return None

In [55]:
genes_bool = np.load('trained_data_2/genes_bool.npy')
original_meta_arr = np.load('trained_data_2/trained_meta_celltypes_HVG.npy')
studies_idx_arr = np.load('trained_data_2/studies_idx_arr.npy')

In [151]:
x, y = np.unique(original_meta_arr, return_counts=True)
x = x[np.argsort(y)[::-1]]
x = [TERM_MAPPING['idx2name'][i] for i in x]
y = y[np.argsort(y)[::-1]]
df = pd.DataFrame({'ct': x, 'count': y})
df

,ct,count
0,Unassigned,2485435
1,T cell,2108257
2,neural cell,966566
3,myeloid leukocyte,885123
4,glial cell,648023
5,B cell,572953
6,epithelial cell,343735
7,connective tissue cell,215157
8,innate lymphoid cell,214594
9,fibroblast,194238


In [ ]:
filtered_indptr = np.load('trained_data_2/filtered_indptr.npy')
filtered_indices = np.load('trained_data_2/filtered_indices.npy')
filtered_data = np.load('trained_data_2/filtered_data.npy')

In [ ]:
for i in range(len(filtered_indptr) - 1):
    start = filtered_indptr[i]
    end = filtered_indptr[i + 1]
    tmp_data = filtered_data[start : end].copy()
    tmp_n_genes = len(tmp_data)
    tmp_sort = np.argsort(tmp_n_genes)
    replaced_data = np.zeros(shape=tmp_n_genes)
    replaced_data[- np.minimum(tmp_n_genes, 5000):] = np.arange(1, 5001)[- np.minimum(tmp_n_genes, 5000):]
    filtered_data[start : end] = replaced_data

In [ ]:
final_matrix = scipy.sparse.csr_matrix((filtered_data, filtered_indices, filtered_indptr), shape=(5033871, 12491))
final_matrix = final_matrix[np.nonzero(meta != 0)[0], :]

In [56]:
meta = original_meta_arr[original_meta_arr != 0]

In [57]:
class_idx, class_count = np.unique(meta, return_counts=True)
class_weight_dct = {class_idx[i]: len(meta) / (len(class_idx) * class_count[i]) for i in range(len(class_idx))}

In [74]:
import numpy as np
from sklearn import linear_model
clf = linear_model.SGDClassifier(
    loss='modified_huber',
    random_state=100,
    class_weight=class_weight_dct
)

In [61]:
TRAINED_STUDIES

['1640',
 '1647',
 '1668',
 '1671',
 '1682',
 '1683',
 '1694',
 '1697',
 '1699',
 '1703',
 '1725',
 '1731',
 '1736',
 '1749',
 '1752',
 '1782',
 '1799',
 '1810',
 '1821',
 '1834',
 '1842',
 '1848',
 '1853',
 '1884',
 '1888',
 '1915',
 '1958',
 '1962',
 '1975',
 '1977',
 '1980',
 '2003',
 '2010',
 '2025',
 '2054',
 '2058',
 '2071',
 '2085',
 '2088',
 '2136',
 '2142',
 '2155',
 '2167',
 '2175',
 '2207',
 '2232',
 '2234',
 '2236',
 '2238',
 '2243',
 '2249',
 '2260',
 '2261',
 '2274',
 '2280',
 '2281',
 '2298',
 '2311',
 '2334',
 '2344',
 '2360',
 '2377',
 '2379',
 '2394',
 '2400',
 '2411',
 '2413',
 '2448',
 '2453',
 '2472',
 '2475',
 '2487',
 '2517',
 '2518',
 '2532',
 '2541',
 '2545',
 '2572',
 '2576',
 '2580',
 '2598',
 '2607',
 '2615',
 '2630',
 '2647',
 '2649',
 '2694',
 '2700',
 '2706',
 '2713',
 '2720',
 '2733',
 '2734',
 '2744',
 '2755',
 '2762',
 '2775',
 '2797',
 '2803',
 '2804',
 '2819',
 '2827',
 '2846',
 '2849',
 '2870',
 '2880',
 '2892',
 '2893',
 '2897',
 '2904',
 '2912',
 

In [67]:
class_idx

array([   6,  108,  193,  278,  342,  359,  387,  413,  490,  613,  616,
        642,  754,  799,  826,  861,  977,  993,  997, 1113, 1579, 1580,
       1807, 1814, 1874, 1988, 1994], dtype=uint16)

In [76]:
hdf5_path = os.path.join(ROOT_DIR, 'camel_2', str(2706), 'raw.hdf5')
k = h5py.File(hdf5_path)

In [77]:
k['expression'].keys()

<KeysViewHDF5 ['indices', 'indptr', 'lognorm']>

In [79]:
TRAINED_STUDIES.index('2706')

88

In [75]:
for i in range(len(TRAINED_STUDIES)):
    study_idx = TRAINED_STUDIES[i]
    
    print(study_idx, '- start')
    
    tmp_meta_arr = original_meta_arr[studies_idx_arr == i]
    tmp_cell_bool = tmp_meta_arr > 0
    if np.sum(tmp_cell_bool) == 0:
        print ('!!!!!WARNING: No metadata, skip study:', study_idx)
        continue
    
    hdf5_path = os.path.join(ROOT_DIR, 'camel_2', str(study_idx), 'raw.hdf5')
    with h5py.File(hdf5_path) as f:
        bc = f['barcodes'][()]
        fe = f['features'][()]
        indptr = f['expression']['indptr'][()].astype(np.int32)
        indices = f['expression']['indices'][()].astype(np.int32)
        try:
            raw = f['expression']['raw'][()].astype(np.float32)
        except:
            raw = f['expression']['lognorm'][()].astype(np.float32)

    print(study_idx, '- create_matrix')
    tmp_mtx = scipy.sparse.csr_matrix((raw, indices, indptr), shape=(len(fe), len(bc)))
    tmp_mtx = tmp_mtx.T.tocsr()

    print(study_idx, '- lognormalize matrix')
    is_raw_matrix = False
    if (not np.all(raw.astype('int') == raw) and np.sum(raw > 30) > 0) or np.all(raw.astype('int') == raw):
        is_raw_matrix = True

    if is_raw_matrix:
        tmp_mtx = normalize_total(tmp_mtx)
    else:
        tmp_mtx = convert_library_size(tmp_mtx)

    if tmp_mtx is None:
        print ('!!!!!WARNING: Cannot lognormalize, Skip study:', study_idx)
        continue
    # print (tmp_mtx.shape)
    # print (np.nonzero(tmp_cell_bool)[0])
    # print (np.nonzero(genes_bool)[0])
    tmp_mtx = tmp_mtx[
        np.nonzero(tmp_cell_bool)[0],
        :
    ]
    tmp_mtx = tmp_mtx[
        :,
        np.nonzero(genes_bool)[0]
    ]
    # print (np.unique(tmp_meta_arr[tmp_cell_bool]))
    print(study_idx, '- partial_fit')
    clf.partial_fit(
        tmp_mtx, 
        tmp_meta_arr[tmp_cell_bool], 
        classes=class_idx
    )

1640 - start
1640 - create_matrix
1640 - lognormalize matrix
1640 - partial_fit
1647 - start
1647 - create_matrix
1647 - lognormalize matrix
1647 - partial_fit
1668 - start
1668 - create_matrix
1668 - lognormalize matrix
1668 - partial_fit
1671 - start
1671 - create_matrix
1671 - lognormalize matrix
1671 - partial_fit
1682 - start
1682 - create_matrix
1682 - lognormalize matrix
1682 - partial_fit
1683 - start
1683 - create_matrix
1683 - lognormalize matrix
1683 - partial_fit
1694 - start
1694 - create_matrix
1694 - lognormalize matrix
1694 - partial_fit
1697 - start
1697 - create_matrix
1697 - lognormalize matrix
1697 - partial_fit
1699 - start
1699 - create_matrix
1699 - lognormalize matrix
1699 - partial_fit
1703 - start
1703 - create_matrix
1703 - lognormalize matrix
1703 - partial_fit
1725 - start
!!!!!WARNING: No metadata, skip study: 1725
1731 - start
1731 - create_matrix
1731 - lognormalize matrix
1731 - partial_fit
1736 - start
1736 - create_matrix
1736 - lognormalize matrix
17

KeyError: "Unable to synchronously open object (object 'data' doesn't exist)"

In [80]:
for i in range(len(TRAINED_STUDIES)):
    if i < 88:
        continue

    study_idx = TRAINED_STUDIES[i]
    
    print(study_idx, '- start')
    
    tmp_meta_arr = original_meta_arr[studies_idx_arr == i]
    tmp_cell_bool = tmp_meta_arr > 0
    if np.sum(tmp_cell_bool) == 0:
        print ('!!!!!WARNING: No metadata, skip study:', study_idx)
        continue
    
    hdf5_path = os.path.join(ROOT_DIR, 'camel_2', str(study_idx), 'raw.hdf5')
    with h5py.File(hdf5_path) as f:
        bc = f['barcodes'][()]
        fe = f['features'][()]
        indptr = f['expression']['indptr'][()].astype(np.int32)
        indices = f['expression']['indices'][()].astype(np.int32)
        try:
            raw = f['expression']['raw'][()].astype(np.float32)
        except:
            raw = f['expression']['lognorm'][()].astype(np.float32)

    print(study_idx, '- create_matrix')
    tmp_mtx = scipy.sparse.csr_matrix((raw, indices, indptr), shape=(len(fe), len(bc)))
    tmp_mtx = tmp_mtx.T.tocsr()

    print(study_idx, '- lognormalize matrix')
    is_raw_matrix = False
    if (not np.all(raw.astype('int') == raw) and np.sum(raw > 30) > 0) or np.all(raw.astype('int') == raw):
        is_raw_matrix = True

    if is_raw_matrix:
        tmp_mtx = normalize_total(tmp_mtx)
    else:
        tmp_mtx = convert_library_size(tmp_mtx)

    if tmp_mtx is None:
        print ('!!!!!WARNING: Cannot lognormalize, Skip study:', study_idx)
        continue
    # print (tmp_mtx.shape)
    # print (np.nonzero(tmp_cell_bool)[0])
    # print (np.nonzero(genes_bool)[0])
    tmp_mtx = tmp_mtx[
        np.nonzero(tmp_cell_bool)[0],
        :
    ]
    tmp_mtx = tmp_mtx[
        :,
        np.nonzero(genes_bool)[0]
    ]
    # print (np.unique(tmp_meta_arr[tmp_cell_bool]))
    print(study_idx, '- partial_fit')
    clf.partial_fit(
        tmp_mtx, 
        tmp_meta_arr[tmp_cell_bool], 
        classes=class_idx
    )

2706 - start
2706 - create_matrix
2706 - lognormalize matrix
2706 - partial_fit
2713 - start
2713 - create_matrix
2713 - lognormalize matrix
2713 - partial_fit
2720 - start
2720 - create_matrix
2720 - lognormalize matrix
2720 - partial_fit
2733 - start
2733 - create_matrix
2733 - lognormalize matrix
2733 - partial_fit
2734 - start
2734 - create_matrix
2734 - lognormalize matrix
2734 - partial_fit
2744 - start
2744 - create_matrix
2744 - lognormalize matrix
2744 - partial_fit
2755 - start
2755 - create_matrix
2755 - lognormalize matrix
2755 - partial_fit
2762 - start
2762 - create_matrix
2762 - lognormalize matrix
2762 - partial_fit
2775 - start
2775 - create_matrix
2775 - lognormalize matrix
2775 - partial_fit
2797 - start
2797 - create_matrix
2797 - lognormalize matrix
2797 - partial_fit
2803 - start
2803 - create_matrix
2803 - lognormalize matrix
2803 - partial_fit
2804 - start
2804 - create_matrix
2804 - lognormalize matrix
2804 - partial_fit
2819 - start
2819 - create_matrix
2819 -

In [ ]:
studies_idx_arr = []
for i in range(len(TRAINED_STUDIES)):
    study_idx = TRAINED_STUDIES[i]
    hdf5_path = os.path.join(ROOT_DIR, 'camel', str(study_idx), 'raw.hdf5')
    with h5py.File(hdf5_path) as f:
        bc = f['barcodes'][()]
    tmp = np.zeros(shape=len(bc))
    tmp[:] = i
    studies_idx_arr.append(tmp)

In [ ]:
studies_idx_arr = np.concatenate(studies_idx_arr)
studies_idx_arr = studies_idx_arr.astype('int')
studies_idx_arr = studies_idx_arr[np.nonzero(meta != 0)[0]]

In [ ]:
# studies_idx_arr = np.concatenate(studies_idx_arr)
# studies_idx_arr = studies_idx_arr.astype('int')
# np.save('trained_data_2/studies_idx_arr.npy', studies_idx_arr)

In [ ]:
studies_idx_arr

In [ ]:
np.array(TRAINED_STUDIES)

In [ ]:
for i in np.unique(studies_idx_arr):
    print (i)
    tmp_bool = np.nonzero(studies_idx_arr == i)[0]
    tmp_matrix = final_matrix[tmp_bool, :] 
    tmp_matrix.indptr = tmp_matrix.indptr.astype(np.int32)
    tmp_matrix.indices = tmp_matrix.indices.astype(np.int32)
    tmp_meta = meta[tmp_bool]
    clf.partial_fit(tmp_matrix, tmp_meta, classes=np.unique(meta))

In [82]:
import pickle

filename = 'trained_data_2/SGD_partial_fit_study_batch_HVG_meta_weight_modified_huber.sav'
pickle.dump(clf, open(filename, 'wb'))

In [83]:
clf

SGDClassifier(class_weight={6: 0.43178966718942946, 108: 0.11734583837984894,
                            193: 18.221638446283066, 278: 1.149835632515722,
                            342: 1.992920605340754, 359: 0.7197264904219389,
                            387: 1.2736703692644342, 413: 0.3817691427390466,
                            490: 1.152852294030519, 613: 9.127626371944555,
                            616: 0.2795037358482213, 642: 0.2559527080253032,
                            754: 6.098133678059237, 799: 4.853550678513403,
                            826: 2.7236267125954794, 861: 254.26021087891593,
                            977: 33.4679633637967, 993: 8.636290762591118,
                            997: 16.675329279130843, 1113: 50.39624876455188,
                            1579: 9.77692005948408, 1580: 4.276715908952672,
                            1807: 87.82221696314703, 1814: 252.9603120502916,
                            1874: 6.527577445519398, 1988: 219.32197268190177,
                            1994: 224.4965382805673},
              loss='modified_huber', random_state=100)

In [ ]:
clf = pickle.load(open('trained_data_2/SGD_partial_fit_study_batch_HVG_meta_weight_class.sav', 'rb'))

In [ ]:
clf

In [227]:
with h5py.File('../inference_dir/2979/raw.hdf5') as f:
    indptr = f['expression']['indptr'][()].astype(np.int32)
    indices = f['expression']['indices'][()].astype(np.int32)
    data = f['expression']['raw'][()]
    bc = f['barcodes'][()]
    fe = f['features'][()]

In [231]:
mtx = scipy.sparse.csr_matrix((data, indices, indptr), shape=(len(fe), len(bc))).T.tocsr()

In [172]:
mtx = convert_library_size(mtx)

In [217]:
mtx = normalize_total(mtx)

In [233]:
mtx = mtx[:, :N_GENES][:, np.nonzero(genes_bool)[0]]

In [234]:
res = clf.predict(mtx)

In [235]:
res

array([359, 359, 359, ..., 108, 108, 108], dtype=uint16)

In [236]:
proba_res = clf.predict_proba(mtx)

In [237]:
for i in range(len(res)):
    ct_idx = res[i]
    arr_idx = np.nonzero(clf.classes_ == ct_idx)[0]
    if proba_res[i, arr_idx] < 0.99:
        res[i] = 0

In [238]:
import pandas as pd

In [239]:
TERM_MAPPING = read_compressed_json('/home/ub-sonvo-25d094476064960/term_mapping.json.gz')

In [240]:
res = [TERM_MAPPING['idx2name'][i] for i in res]
res

['epithelial cell',
 'epithelial cell',
 'epithelial cell',
 'endothelial cell',
 'T cell',
 'epithelial cell',
 'epithelial cell',
 'epithelial cell',
 'epithelial cell',
 'epithelial cell',
 'epithelial cell',
 'endothelial cell',
 'epithelial cell',
 'epithelial cell',
 'endothelial cell',
 'endothelial cell',
 'epithelial cell',
 'T cell',
 'epithelial cell',
 'epithelial cell',
 'epithelial cell',
 'epithelial cell',
 'Unassigned',
 'endothelial cell',
 'epithelial cell',
 'Unassigned',
 'epithelial cell',
 'epithelial cell',
 'epithelial cell',
 'epithelial cell',
 'epithelial cell',
 'epithelial cell',
 'epithelial cell',
 'epithelial cell',
 'endothelial cell',
 'T cell',
 'epithelial cell',
 'endothelial cell',
 'epithelial cell',
 'muscle cell',
 'epithelial cell',
 'epithelial cell',
 'T cell',
 'endothelial cell',
 'epithelial cell',
 'epithelial cell',
 'epithelial cell',
 'endothelial cell',
 'epithelial cell',
 'T cell',
 'epithelial cell',
 'epithelial cell',
 'T cell',

In [241]:
import pandas as pd

In [242]:
df = pd.DataFrame({'SGD_partial_fit_study_batch_HVG_modified_huber_0.99_150studies': res}, index=bc.astype('str'))

In [243]:
df.index.name = 'Barcodes'
df.to_csv('test_meta.tsv', sep='\t')

In [ ]:
meta.shape

In [ ]:
np.sum(meta == 616) / len(meta)

In [ ]:
np.sum(meta == 342) / len(meta)

In [ ]:
np.sum(meta == 387) / len(meta)

In [ ]:
x, y = np.unique(meta, return_counts=True)
x = x[np.argsort(y)]
y = y[np.argsort(y)]

In [ ]:
for i in range(len(x)):
    print (TERM_MAPPING['idx2name'][x[i]], y[i] / len(meta))

In [ ]:
y 

____

## transfer to rank, visualize expression

In [ ]:
fibroblast_bool = meta == 387
fibroblast_idx = np.nonzero(fibroblast_bool)[0]

In [ ]:
final_matrix.shape

In [ ]:
fibroblast_matrix = final_matrix[fibroblast_idx, :]

In [ ]:
studies_idx_arr = studies_idx_arr[fibroblast_bool]

In [ ]:
t = []
for i in [1372, 5778, 3683, 2246]:
    for study_idx in np.unique(studies_idx_arr):
        study_bool_idx = np.nonzero(studies_idx_arr==study_idx)[0]
        study_count = len(study_bool_idx)
        study_sum = np.sum(fibroblast_matrix[study_bool_idx, i].todense())
        study_average = study_sum / study_count
        t.append((i, study_idx, study_average))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
x = []
y = []
for k in t:
    if k[0] == 1372:
        x.append(k[1])
        y.append(k[2])
fig, ax = plt.subplots()
ax.plot(x, y, 'ro')
plt.show()

In [ ]:
x = []
y = []
for k in t:
    if k[0] == 5778:
        x.append(k[1])
        y.append(k[2])
fig, ax = plt.subplots()
ax.plot(x, y, 'ro')
plt.show()

In [ ]:
x = []
y = []
for k in t:
    if k[0] == 3683:
        x.append(k[1])
        y.append(k[2])
fig, ax = plt.subplots()
ax.plot(x, y, 'ro')
plt.show()

In [ ]:
x = []
y = []
for k in t:
    if k[0] == 2246:
        x.append(k[1])
        y.append(k[2])
fig, ax = plt.subplots()
ax.plot(x, y, 'ro')
plt.show()

In [ ]:
x = []
y = []
for k in t:
    if k[0] == 1372:
        x.append(k[1])
        y.append(k[2])
fig, ax = plt.subplots()
ax.plot(x, y, 'ro')
plt.show()

In [ ]:
list(zip(np.unique(res, return_counts=True)))

In [ ]:
res

In [ ]:
res.shape

In [ ]:
print ('a')